In [2]:
# https://docs.google.com/document/d/1e9oakyEftdHp4zGz2F0WP-_X7GEzEKYGaPvu1SguuCE/edit

import pandas as pd
import numpy as np
import matplotlib as plt

# https://pypi.org/project/yfinance/#description
import yfinance as yf

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [3]:
portfolio = ['MSFT', 'AAPL', 'IBM', 'WMT', 'SHOP']
df = yf.download(portfolio,'2022-1-1') # Pulling the daily closing stock price for the aforementioned stocks from the start of 2022-onwards.

[*********************100%***********************]  5 of 5 completed


In [4]:
print(df.tail())

             Adj Close                                                       Close              \
                  AAPL         IBM        MSFT        SHOP         WMT        AAPL         IBM   
Date                                                                                             
2022-02-17  168.880005  124.970001  290.730011  660.000000  138.880005  168.880005  124.970001   
2022-02-18  167.300003  124.349998  287.929993  656.880005  137.990005  167.300003  124.349998   
2022-02-22  164.320007  123.919998  287.720001  626.159973  136.449997  164.320007  123.919998   
2022-02-23  160.070007  122.070000  280.269989  626.609985  135.050003  160.070007  122.070000   
2022-02-24  155.929993  120.089996  278.489197  611.114990  133.029999  155.929993  120.089996   

                                                      High                                      \
                  MSFT        SHOP         WMT        AAPL         IBM        MSFT        SHOP   
Date               

### Portfolio of 5 stocks is now present. Data is daily changes, from the start of the year, and contains 6 features.

# Transformations:

### Transform 1: Group by industry

Calculate:
* Market value per industry (dollar amount) for the day
* % change of market value from previous day for each industry

(market value = adjusted close * volume)

In [18]:
a = yf.Ticker("MSFT").info

In [21]:
a['sector']

'Technology'

### Transform 1: % change of previous day close price vs current day close price

In [8]:
# Below we'll calculate the percent change of Adj Close price for each stock and, if the price change for one exceeds a predermined threshold, an email notification will be sent out.
last_two_days = df['Adj Close'].iloc[-2:].pct_change()
percent_change = last_two_days.iloc[-1]
percent_change

AAPL   -0.025864
IBM    -0.016220
MSFT   -0.006354
SHOP   -0.024728
WMT    -0.014957
Name: 2022-02-24 00:00:00, dtype: float64

### Transform 1: % change of previous day close price vs current day close price